<table style='border: none' align='left'>
   <tr style='border: none'>
      <th style='border: none'><font face='verdana' size='5' color='black'><b>Deploy a Shiny App using the WML Python client</b></th>
      <th style='border: none'><img src='https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true' alt='Watson Machine Learning icon' height='40' width='40'></th>
   </tr>
</table>

This notebook catalogs the steps needed to deploy a Shiny app using the Watson Machine Learning Python client. It's compatible with CP4D 3.0 and Python 3.6. 

R's <a href="https://shiny.rstudio.com/" target="_blank" rel="noopener noreferrer">Shiny</a> package gives users the ability to view interactive visualizations displayed using web applications. The components of a Shiny app are built using an <a href="https://www.r-project.org/about.html" target="_blank" rel="noopener noreferrer">R script</a>. So, when the user interacts with the input displayed in the web application, the Shiny app reacts to those changes, reruns the R Script with the new input, and rerenders the new results in the visualization.

The R script used to create a Shiny app should have the following components:
- **The UI object** - This generates the HTML that displays the app.
- **The Server function** - This is the code that generates the visualization that the user will be able to interact with.

You can see the R script used in the Hello Shiny example below. This script will be used for Shiny deployment in this notebook example.

```

library(shiny)
 
# Define UI for app that draws a histogram ----
ui <- fluidPage(

  # App title ----
  titlePanel("Hello Shiny!"),

  # Sidebar layout with input and output definitions ----
  sidebarLayout(

    # Sidebar panel for inputs ----
    sidebarPanel(

      # Input: Slider for the number of bins ----
      sliderInput(inputId = "bins",
                  label = "Number of bins:",
                  min = 1,
                  max = 50,
                  value = 30)
    ),

    # Main panel for displaying outputs ----
    mainPanel(

      # Output: Histogram ----
      plotOutput(outputId = "distPlot")
    )
  )
)

# Define server logic required to draw a histogram ----
server <- function(input, output) {

  # Histogram of the Old Faithful Geyser Data ----
  # with requested number of bins
  # This expression that generates a histogram is wrapped in a call
  # to renderPlot to indicate that:
  #
  # 1. It is "reactive" and therefore should be automatically
  #    re-executed when inputs (input$bins) change
  # 2. Its output type is a plot
  output$distPlot <- renderPlot({

    x    <- faithful$waiting
    bins <- seq(min(x), max(x), length.out = input$bins + 1)

    hist(x, breaks = bins, col = "#75AADB", border = "white",
         xlab = "Waiting time to next eruption (in mins)",
         main = "Histogram of waiting times")

    })

}

# Create Shiny app ----
shinyApp(ui = ui, server = server)
```

Here, you'll see the UI component that defines the title of the webpage, the sidebar with the input slider (for user interaction), and the panel where the visualization/plot will be displayed.

This app uses data from R's built-in dataset <a href="https://www.rdocumentation.org/packages/mixAK/versions/5.1/topics/Faithful" target="_blank" rel="noopener noreferrer">`faithful`</a>, as you can see in the server component. The dataset deals with eruptions of the Old Faithful Geyser in Yellowstone National Park. The histogram rendered on the webpage is defined in the server component – you can see that as the user controls the slider, the input is stored in the variable `input$bins`.

Finally, the `shinyApp` function uses the UI and server components to create the Shiny App, which should look like this:

![ShinyWebApp](https://github.com/IBMDataScience/sample-notebooks/blob/master/Files/ShinyWebApp.png?raw=true)

## Table of Contents

This notebook contains the following parts:
1. [Set up the environment](#setup)
2. [Create a Shiny asset](#shinyasset)
3. [Deploy the Shiny app](#deploy)
4. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [2]:
# Enter your credentials here
import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

from project_lib.utils import environment
url = environment.get_common_api_url()

wml_credentials = {
"token": token,
"instance_id" : "wml_local",
"url": "***",
"version": "3.0.0"
}

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.64'

### To set the default space, follow these steps.

In order to deploy a Shiny app, you'll need to have a deployment space.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [5]:
# Obtain the UID of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [6]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = 594b22d1-588c-4de0-9cbd-d98e6fa890e8


You can set the default space using the cell below.

In [7]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="shinyasset"></a>
## 2. Create a Shiny asset

To create a shiny asset & deploy the shiny app, you need to have a compressed R zip file ready. The R file should be named `app.R` in order to render the interface properly. You can download it below.

In [8]:
!wget https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/app.R.zip --output-document=app.R.zip

--2020-05-12 04:21:05--  https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/app.R.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/app.R.zip [following]
--2020-05-12 04:21:06--  https://raw.githubusercontent.com/IBMDataScience/sample-notebooks/master/Files/app.R.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1251 (1.2K) [application/zip]
Saving to: ‘app.R.zip’

100%[======================================>] 1,251       --.-K/s   in 0s      

2020-05-12 04:21:06 (16.7 MB/s) - ‘app.R.zip’ saved [1251/1251]



Create the following metadata to store the zip file as an asset in your deployment space

In [9]:
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: 'My_Shiny_App',
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'My_Shiny_App description' # optional
}

In [10]:
# create the asset for the shiny app
app_details = client.shiny.store(meta_props, 'app.R.zip')

Creating Shiny asset...
SUCCESS


List the assets using the following code.

In [11]:
client.shiny.list()

------------  -----------  ------------------------------------
NAME          ASSET_TYPE   ASSET_ID
My_Shiny_App  shiny_asset  f27be506-7389-45a7-8b25-e8cd8574e3b2
------------  -----------  ------------------------------------


<div class="alert alert-block alert-info">
From the list of assets, you can see that the shiny asset was successfully created in the deployment space.</div>

Get the UID and the asset details.

In [12]:
app_uid = client.shiny.get_uid(app_details)
print('App UID = {}'.format(app_uid))
print('App: {}'.format(client.shiny.get_details(app_uid)))

App UID = f27be506-7389-45a7-8b25-e8cd8574e3b2
App: {'metadata': {'space_id': '594b22d1-588c-4de0-9cbd-d98e6fa890e8', 'guid': 'f27be506-7389-45a7-8b25-e8cd8574e3b2', 'href': '/v2/assets/f27be506-7389-45a7-8b25-e8cd8574e3b2?space_id=594b22d1-588c-4de0-9cbd-d98e6fa890e8', 'asset_type': 'shiny_asset', 'created_at': '2020-05-12T04:21:08Z', 'last_updated_at': '2020-05-12T04:21:09Z'}, 'entity': {}}


<a id="deploy"></a>
## 2. Deploy the Shiny app

Now, you can create the deployment metadata.

In [13]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: 'My_Shiny_App_deployment',
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'My_Shiny_App_deployment description',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

In [14]:
# Create the deployment.
deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: 'f27be506-7389-45a7-8b25-e8cd8574e3b2' started

#######################################################################################


initializing......................................................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='195c2bf6-6421-455e-afc0-592a591dafbf'
------------------------------------------------------------------------------------------------




Get the list of all deployments.

In [15]:
client.deployments.list()

------------------------------------  ------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                  STATE  CREATED                   ARTIFACT_TYPE
195c2bf6-6421-455e-afc0-592a591dafbf  My_Shiny_App_deployment               ready  2020-05-12T04:21:11.788Z  unknown
02d47cec-562c-4f63-8d87-f47c55246775  Sample SPSS model deployment          ready  2020-05-12T04:04:27.203Z  model
4de63cb0-974e-4a1f-8e6d-2ef18119528a  Sample SPSS model deployment          ready  2020-05-12T03:56:56.003Z  model
ec0c61f1-9134-48f4-a4e5-0feaa0efc60e  Tensorflow custom library deployment  ready  2020-05-12T03:33:51.111Z  model
------------------------------------  ------------------------------------  -----  ------------------------  -------------


<div class="alert alert-block alert-info">
From the list of deployments, you can see that Shiny app was successfully deployed.</div>

In [ ]:
import pprint as pp
deployment_uid = client.deployments.get_uid(deployment)
print('Deployment UID = {}'.format(deployment_uid))
print('Shiny app deployment:\n{}'.format(pp.pformat(deployment)))

You can see your Shiny app deployed at the following URL.

In [ ]:
# This is the URL where the shiny app is deployed.
print('The Shiny app deployment is created at: {}'.format(deployment['entity']['status']['rshiny_url']['url']))

You can delete the Shiny deployment and asset using the code below. The cells are not executable at this stage, but you can use them if needed.

## 4. Summary and next steps <a id="summary"></a>

You have successfully completed this notebook!

You learned how to use the Python client and Watson Machine Learning to deploy a Shiny app.

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-shiny-app.html" target="_blank" rel="noopener noreferrer">Creating a Shiny app deployment</a></li>
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/deploy-shiny-app-python.html" target="_blank" rel="noopener noreferrer">Deploying Shiny apps using Python</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<ul>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 <li><a href="https://shiny.rstudio.com/" target="_blank" rel="noopener noreferrer">R Shiny</a></li>
 <ul>
     <li><a href="https://shiny.rstudio.com/tutorial/" target="_blank" rel="noopener noreferrer">R Shiny Video Tutorial</a></li>
     <li><a href="https://shiny.rstudio.com/articles/" target="_blank" rel="noopener noreferrer">Shiny Articles for Learning</a></li>
 </ul>
</ul>
 </div>

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style='background:#F5F7FA; height:110px; padding: 2em; font-size:14px;'>
<span style='font-size:18px;color:#152935;'>Love this notebook? </span>
<span style='font-size:15px;color:#152935;float:right;margin-right:40px;'>Don't have an account yet?</span><br>
<span style='color:#5A6872;'>Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style='border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;'><a href='https://ibm.co/wsnotebooks' target='_blank' style='color: #3d70b2;text-decoration: none;'>Sign Up</a></span><br>
</div>